## Useful Knowledge Base to familiar with some core conepts useful for this project

### Overview

[IAM Roles] (https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles.html#id_roles_terms-and-concepts)

[How to Trust Policies with IAM Roles] (https://aws.amazon.com/blogs/security/how-to-use-trust-policies-with-iam-roles/)

[IAM Permission for AgentCore RunTime] (https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-permissions.html)

[Prerequisites to Set Up a Bedrock AgentCore Gateway] (https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway-prerequisites-permissions.html)

[Consolidate permission set in Github to enable the role SmartGoalGeneratorBedrockAgentCoreRole-<REGION>] (https://github.com/aws/bedrock-agentcore-starter-toolkit/blob/main/documentation/docs/user-guide/runtime/permissions.md

[Invoke an AgentCore Runtime agent --- MUST READ for mutli-agent deployment in AgentCore] (https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-invoke-agent.html)
)

### Additional references
[Strands Agents] (https://strandsagents.com/latest/documentation/docs/user-guide/concepts/multi-agent/agent-to-agent/)

[Bedrock Agents Healthcare Life Sciences Use Case] (https://github.com/aws-samples/amazon-bedrock-agents-healthcare-lifesciences)

[Bedrock Agentcore Development Guide] (https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-invoke-agent.html)

[Prescriptive Guidance for Agentic AI] (https://docs.aws.amazon.com/prescriptive-guidance/latest/agentic-ai-frameworks/comparing-agentic-ai-frameworks.html)

[MCP Concept] (https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/mcp-tools/)

## Phase 1: Creating a smart goal generator agent prototype

### Overview

[Amazon Bedrock AgentCore](https://aws.amazon.com/bedrock/agentcore/) helps you deploying and operating AI agents securely at scale - using any framework and model. It provides you with the capability to move from prototype to production faster. 

In this 5-labs tutorial, we will demonstrate the end-to-end journey from prototype to production using a **Smart Goal Generator Agent**. For this example we will use [Strands Agents](https://strandsagents.com/latest/), a simple-to-use, code-first framework for building agents and the Anthropic Claude Sonnet 3.7 model from Amazon Bedrock. For your application you can use the framework and model of your choice. It's important to note that the concepts covered here can be applied using other frameworks and models as well.

**Staging Phases:**
- **Phase 1**: Create Agent Prototype - Build a functional Smart Goal Generator agent
- **Phase 2**: Scale with Gateway & Identity - Share tools across agents securely
- **Phase 3**: Deploy to Production - Use AgentCore Runtime with observability
- **Phase 4**: Build User Interface - Create a customer-facing application

In this first lab, we'll build a Smart Goal Generator Agent prototype that will evolve throughout the workshop into a production-ready system serving multiple customers with persistent memory, shared tools, and full observability. Our agent will have the following local tools available:
- **fetch_data()** - Get patient data from data sources 

### Architecture for Phase 1
<div style="text-align:left">
    <img src="images/architecture_lab1_strands.png" width="75%"/>
</div>

*Simple prototype running locally. In subsequent labs, we'll migrate this to AgentCore services with shared tools, persistent memory, and production-grade observability.*

### Prerequisites

* **AWS Account** with appropriate permissions
* **Python 3.10+** installed locally
* **AWS CLI configured** with credentials
* **Anthropic Claude 3.7** enabled on [Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)
* **Strands Agents** and other libraries installed in the next cells
#### Not using an AWS workshop account? 

**Note:** If you are running this as a self-paced lab you must run create the cloudformation resources as shown in the workshop self-paced steps. If you have not, then uncomment and run the below code segment

In [1]:
!bash scripts/prereq.sh

🔍 Getting AWS Account ID...
Region: us-east-1
Account ID: 711246752798
🪣 Using S3 bucket: smartgoalgenerator-711246752798-us-east-1
{
    "Location": "/smartgoalgenerator-711246752798-us-east-1"
}
🔍 Verifying S3 bucket ownership...
{
    "BucketRegion": "us-east-1",
    "AccessPointAlias": false
}
✅ S3 bucket ownership verified
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
zip is already the newest version (3.0-12build2).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
📦 Zipping contents of prerequisite/lambda/python into lambda.zip...
☁️ Uploading lambda.zip to s3://smartgoalgenerator-711246752798-us-east-1/lambda.zip...
{
    "ETag": "\"c0f6b02ffb4a6075c93cbd9cc871ca9d\"",
    "ChecksumCRC64NVME": "0LAUjODvwU4=",
    "ChecksumType": "FULL_OBJECT",
    "ServerSideEncryption": "AES256"
}
☁️ Uploading ddgs-layer.zip to s3://smartgoalgenerator-711246752798-us-east-1/ddgs-layer.zip...
{
    "ETag": "\"0f2b60895990988b94f725

In [2]:
# Install required packages
%pip install -U -r requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import libraries
import boto3
from boto3.session import Session

from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

from strands.tools import tool
from lab_helpers.utils import put_ssm_parameter

In [4]:
# Get boto session
boto_session = Session()
region = boto_session.region_name

## Phase 1 - Step 1: Install and import required libraries

In [5]:
# Refer here for the IAM required for the runtime permission
# https://github.com/aws/bedrock-agentcore-starter-toolkit/blob/main/documentation/docs/user-guide/runtime/permissions.md

# Install required packages
%pip install strands-agents "boto3>=1.39.15" strands-agents-tools bedrock_agentcore ddgs -q

Note: you may need to restart the kernel to use updated packages.


In [6]:
# Import libraries
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
import os
import sys
import boto3
import json
from bedrock_agentcore.identity.auth import requires_access_token
from mcp.client.streamable_http import streamablehttp_client
import requests

from scripts.utils import get_ssm_parameter, put_ssm_parameter, load_api_spec, get_cognito_client_secret

sts_client = boto3.client('sts')

# Get AWS account details
REGION = boto3.session.Session().region_name

gateway_client = boto3.client(
    "bedrock-agentcore-control",
    region_name=REGION,
)

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


## Phase 1 - Step 2: Give our agent a tool to access existing customer data
AgentCore Gateway simplifies agent tool integration in three key ways:

Universal MCP Support: Instantly make your tools compatible with any agent framework by exposing them through AgentCore Gateway's MCP standard

Simple REST Integration: Transform existing REST services into agent tools by just adding them as AgentCore Gateway targets

Lambda Flexibility: Expose Lambda functions as MCP endpoints that can call any API - demonstrated here with a function that checks warranty status

AgentCore Gateway populates the Lambda context with the name of the tool to invoke, while the parameters passed to the tool are provided in the Lambda event:

```
extended_tool_name = context.client_context.custom["bedrockAgentCoreToolName"]
resource = extended_tool_name.split("___")[1]
```

[Lambda function](./prerequisite/lambda/python/lambda_function.py)

```
def lambda_handler(event, context):
    if get_tool_name(event) == "check_warranty_status":
        serial_number = get_named_parameter(event=event, name="serial_number")
        customer_email = get_named_parameter(event=event, name="customer_email")

        warranty_status = check_warranty_status(serial_number, customer_email)
        return {"statusCode": 200, "body": warranty_status}
```

## Phase 1 - Step 3 Convert your web search tool to MCP
Now that we are developing an MCP server using AgentCore Gateway, we can MCP-ify any tools which we think we'll use for multiple Agents. One of these tools might be a web search tool like we built in Lab1. As a result, we also converted the web search tool from Lab 1 into a Lambda tool within our AgentCore Gateway:

[fetch_data Lambda](./prerequisite/lambda/python/fetch_data.py)

[Web search Lambda](./prerequisite/lambda/python/web_search.py)



## Phase 1 - Step 4 Create your function definition metadata
Lastly, we need to write tool schema which describes the tools implemented by your Lambda function.

This file has been already defined in [prerequisite/lambda/api_spec.json](./prerequisite/lambda/api_spec.json)


## Phase 2 - Step 1. Create your AgentCore Gateway

Now let's create the AgentCore Gateway to expose the Lambda function as MCP-compatible endpoint.

To validate the callers authorized to invoke our tools we need to configure the Inbound Auth.

Inbound Auth works using OAuth authorization, the standard for MCP servers. With OAuth the client application must authenticate with the OAuth authorizer before using the Gateway. Your client would receive an access token which is used at runtime.

You need to specify an OAuth discovery server and client IDs. The Cloudformation provided with the workshop already provisioned the Cognito UserPool and UserPoolClient and it stored the discovery URL and the Client ID in dedicated SSM parameters.

In [7]:
# ------------- SMART GOAL GENERATOR ---------

gateway_name = "smartgoalgenerator-gw"

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            get_ssm_parameter("/app/smartgoalgenerator/agentcore/machine_client_id")
        ],
        "discoveryUrl": get_ssm_parameter("/app/smartgoalgenerator/agentcore/cognito_discovery_url")
    }
}

try:
    # create new gateway
    print(f"Creating gateway in region {REGION} with name: {gateway_name}")

    create_response = gateway_client.create_gateway(
        name=gateway_name,
        roleArn= get_ssm_parameter("/app/smartgoalgenerator/agentcore/gateway_iam_role"),
        protocolType="MCP",
        authorizerType="CUSTOM_JWT",
        authorizerConfiguration=auth_config,
        description="Smart Goal Generator AgentCore Gateway",
    )

    gateway_id = create_response["gatewayId"]

    gateway = {
        "id": gateway_id,
        "name": gateway_name,
        "gateway_url": create_response["gatewayUrl"],
        "gateway_arn": create_response["gatewayArn"],
    }
    put_ssm_parameter("/app/smartgoalgenerator/agentcore/gateway_id", gateway_id)

    print(f"✅ Gateway created successfully with ID: {gateway_id}")

except Exception as e:
    # If gateway exists, collect existing gateway ID from SSM
    existing_gateway_id = get_ssm_parameter("/app/smartgoalgenerator/agentcore/gateway_id")
    print(f"Found existing gateway with ID: {existing_gateway_id}")
    
    # Get existing gateway details
    gateway_response = gateway_client.get_gateway(gatewayIdentifier=existing_gateway_id)
    gateway = {
        "id": existing_gateway_id,
        "name": gateway_response["name"],
        "gateway_url": gateway_response["gatewayUrl"],
        "gateway_arn": gateway_response["gatewayArn"],
    }
    gateway_id = gateway['id']

Creating gateway in region us-east-1 with name: smartgoalgenerator-gw
Found existing gateway with ID: smartgoalgenerator-gw-kmbshevtab


## Phase 2 - Step 2. Add the Lambda function Target
Now we will use the previously defined function definitions from [prerequisite/lambda/api_spec.json](./prerequisite/lambda/api_spec.json) to create a Lambda target within our Agent Gateway. This will define the tools that your gateway will host.

Gateway allows you to attach multiple targets to a Gateway and you can change the targets / tools attached to a gateway at any point. Each target can have its own credential provider, but Gateway becomes a single MCP URL enabling access to all of the relevant tools for an agent across myriad APIs.

In [9]:
def load_api_spec(file_path: str) -> list:
    with open(file_path, "r") as f:
        data = json.load(f)
        
    if not isinstance(data, list):
        raise ValueError("Expected a list in the JSON file")
    return data

try:
    api_spec_file = "./prerequisite/lambda/api_spec.json"

    # Validate API spec file exists
    if not os.path.exists(api_spec_file):
        print(f"❌ API specification file not found: {api_spec_file}")
        sys.exit(1)

    api_spec = load_api_spec(api_spec_file)
 
    # Use Cognito for Inbound OAuth to our Gateway
    lambda_target_config = {
        "mcp": {
            "lambda": {
                "lambdaArn": get_ssm_parameter("/app/smartgoalgenerator/agentcore/lambda_arn"),
                "toolSchema": {"inlinePayload": api_spec},
            }
        }
    }

    # Create gateway target
    credential_config = [{"credentialProviderType": "GATEWAY_IAM_ROLE"}]

    create_target_response = gateway_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name="LambdaUsingSDK",
        description="Lambda Target using SDK",
        targetConfiguration=lambda_target_config,
        credentialProviderConfigurations=credential_config,
    )

    print(f"✅ Gateway target created: {create_target_response['targetId']}")

except Exception as e:
    print(f"❌ Error creating gateway target: {str(e)}")

✅ Gateway target created: XEPVM3AAH1


## Phase 2 - Step 3: Add our new MCP-based tools to our support agent
Here we integrate our authentication token from Cognito into an MCPClient from Strands SDK to create an MCP Server object to integrate with our Strands Agent

In [10]:
def get_token(client_id: str, client_secret: str, scope_string: str, url: str) -> dict:
    try:
        headers = {"Content-Type": "application/x-www-form-urlencoded"}
        data = {
            "grant_type": "client_credentials",
            "client_id": client_id,
            "client_secret": client_secret,
            "scope": scope_string,

        }
        response = requests.post(url, headers=headers, data=data)
        response.raise_for_status()
        return response.json()

    except requests.exceptions.RequestException as err:
        return {"error": str(err)}

## Phase 2 - Step 4. Set up a secure MCP client object

In [11]:
# ------------- SMART GOAL GENERATOR ---------
gateway_access_token = get_token(
    get_ssm_parameter("/app/smartgoalgenerator/agentcore/machine_client_id"),
    get_cognito_client_secret(),
    get_ssm_parameter("/app/smartgoalgenerator/agentcore/cognito_auth_scope"),
    get_ssm_parameter("/app/smartgoalgenerator/agentcore/cognito_token_url"))

print(f"Gateway Endpoint - MCP URL: {gateway['gateway_url']}")

# Set up MCP client
mcp_client = MCPClient(
    lambda: streamablehttp_client(
        gateway['gateway_url'],
        headers={"Authorization": f"Bearer {gateway_access_token['access_token']}"},
    )
)

Gateway Endpoint - MCP URL: https://smartgoalgenerator-gw-kmbshevtab.gateway.bedrock-agentcore.us-east-1.amazonaws.com/mcp


## Phase 2 - Step 5. Access tools in our agent using our MCP client
Now we will create our Strands Agent using the AgentCore Gateway we built along with the resources from previous labs. Our agent now uses a mix of local tools via our Strands Agent and MCP tools via AgentCore Gateway

In [12]:
#from lab_helpers.smartgoalgenerator_mcp_tools import fetch_data
from lab_helpers.smartgoalgenerator_mcp_tools import load_analyzer_runs_v2,build_eval_plan_v2,fetch_data

#from lab_helpers.lab2_memory import CustomerSupportMemoryHooks,create_or_get_memory_resource 
#import uuid
#from bedrock_agentcore.memory import MemoryClient

# memory_client = MemoryClient(region_name=REGION)

# memory_id = create_or_get_memory_resource()
# SESSION_ID = str(uuid.uuid4())
# CUSTOMER_ID = "customer_001"
# memory_hooks = CustomerSupportMemoryHooks(memory_id, memory_client, CUSTOMER_ID, SESSION_ID)

# Initialize the Bedrock model
# model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
# model = BedrockModel(
#     model_id=model_id,
#     temperature=0.3,  # Balanced between creativity and consistency
#     region_name=REGION
# )

try:
    mcp_client.start()
except Exception as e:
    print(f"Error initializing agent: {str(e)}")

tools = (
            [
                fetch_data,
                load_analyzer_runs_v2,
                build_eval_plan_v2
            ]
            + mcp_client.list_tools_sync()
            # mcp_client has fetch_data and web_search defined in lambda api_spec.json
        )


## Phase 3 - Part 1. Getting Strands and Bedrock libraries for Runtime

In [13]:
# Import required libraries
import os
import json
import boto3
from strands import Agent
from strands.models import BedrockModel

## Phase 3 - CRITICAL Step. Preparing Runtime Code Base

In [14]:
%%writefile ./lab_helpers/smartgoalgenerator_runtime.py
import os
import re
import json
import time
import uuid

import boto3
import json

# ===========================================
# ===== Runtime / Model Imports ============
# ===========================================
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands import Agent
from strands.models import BedrockModel
from scripts.utils import get_ssm_parameter

# Project helpers (must be in your deployment package)
from lab_helpers.smartgoalgenerator_model_util import (
    model_supports_system_prompt,
    model_supports_tools,
    get_analyzer_prompt,
)

# Optional tools
try:
    from lab_helpers.smartgoalgenerator_mcp_tools import (
        load_analyzer_runs_v2,
        build_eval_plan_v2,
        fetch_data,
    )
except Exception:
    load_analyzer_runs_v2 = None
    build_eval_plan_v2 = None
    fetch_data = None

# ===================================
# ============ CONSTANTS ============
# ===================================
MODEL_ID = "mistral.mistral-7b-instruct-v0:2"
#MODEL_ID = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
#MODEL_ID = "meta.llama3-70b-instruct-v1:0"
#MODEL_ID = "mistral.mistral-large-2402-v1:0"
#MODEL_ID = "cohere.command-r-v1:0"
#MODEL_ID = "openai.gpt-oss-120b-1:0"

EVAL_MODEL_ID = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

OUTPUT_DIR_INDIVIDUAL = "./outputs"
output_jsonl = "./outputs/results.jsonl"

# =========================================
# Evaluator runtime ARN
# =========================================
EVALUATOR_RUNTIME_ARN = "arn:aws:bedrock-agentcore:us-east-1:711246752798:runtime/llm_evaluator_agent-noHKUN9oQS"

# =========================================
# Helper function to call evaluator runtime
# =========================================
def call_evaluator_runtime(payload: dict) -> dict:
    # Initialize the Bedrock AgentCore client
    agent_core_client = boto3.client('bedrock-agentcore')
  
    # Prepare the payload prompt
    #payload={'analyzer_payload':output_obj}
    prompt = json.dumps(payload).encode()
  
    # Invoke the agent
    response = agent_core_client.invoke_agent_runtime(
                    agentRuntimeArn=EVALUATOR_RUNTIME_ARN,
                    #agentRuntimeArn="arn:aws:bedrock-agentcore:us-east-1:711246752798:runtime/llm_evaluator_agent-jf0YsKAH8C", 
                    #runtimeSessionId=session_id,
                    payload=prompt
                    )

    # Process and print the response
    if "text/event-stream" in response.get("contentType", ""):
        # Handle streaming response
        content = []
        for line in response["response"].iter_lines(chunk_size=10):
            if line:
                line = line.decode("utf-8")
                if line.startswith("data: "):
                    line = line[6:]
                    print(line)
                    content.append(line)
        #print("\nComplete response:", "\n".join(content))
        return "\n".join(content)
        
    elif response.get("contentType") == "application/json":
        # Handle standard JSON response
        content = []
        for chunk in response.get("response", []):
            content.append(chunk.decode('utf-8'))
        #print(json.loads(''.join(content)))
        return json.loads(''.join(content))
  
    #else:
        # Print raw response for other content types
        #print(response)
    return response


# ===============================================
# ===== Json/Jsonl Utility Helper Functions =====
# ===============================================
def clean_json_str(s: str) -> str:
    # remove trailing commas before } or ]
    s = re.sub(r",\s*([}\]])", r"\1", s)
    # strip any junk after final closing brace
    last_brace = max(s.rfind("}"), s.rfind("]"))
    if last_brace != -1:
        s = s[:last_brace+1]
    return s


def _coerce_json(s):
    import json, re

    if not isinstance(s, str):
        if hasattr(s, "output"): s = s.output
        elif hasattr(s, "content"): s = s.content
        elif hasattr(s, "text"): s = s.text
        else: s = str(s)

    s = s.strip()

    if s.startswith("{") and s.endswith("}"):
        candidate = s
    else:
        m = re.search(r"\{.*\}", s, flags=re.DOTALL)
        if not m:
            raise ValueError("No JSON object found in agent output.")
        candidate = m.group(0)

    candidate = clean_json_str(candidate)
    
    try:
        return json.loads(candidate)
    except json.JSONDecodeError as e:
        # Print useful debug info
        snippet = candidate[max(0, e.pos-80):e.pos+80]
        print(f"\n--- JSON parse error ---\n{e}\nContext:\n...{snippet}...\n")
        raise


def _append_jsonl(path: str, obj: dict):
    os.makedirs(os.path.dirname(path) or ".", exist_ok=True)
    with open(path, "a", encoding="utf-8") as f:
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")


# ===========================================
# ---------- helpers for filenames ----------
# ===========================================
def _basename_no_ext(path_or_uri: str) -> str:
    """
    's3://bucket/path/patient1_summary.docx' -> 'patient1_summary'
    'patient2.pdf' -> 'patient2'
    'https://.../file.txt?x=y' -> 'file' (best effort)
    """
    s = path_or_uri.split("?", 1)[0]
    if s.lower().startswith("s3://"):
        _, key = s[5:].split("/", 1)
        base = os.path.basename(key)
    else:
        base = os.path.basename(s)
    name, _ext = os.path.splitext(base)
    return name or "unknown_source"

def _safe_fragment(s: str) -> str:
    """
    Make a safe filename fragment: replace non [A-Za-z0-9_-] with '_'.
    Also replace ':', '.', '/' commonly found in model ids.
    """
    s = s.replace(":", "_").replace("/", "_").replace(".", "_")
    return "".join(c if c.isalnum() or c in ("-", "_") else "_" for c in s)
    

# =============================================
# ===== Model Selection and configuration =====
# =============================================

# Lab1 import: Create the Bedrock model
#model = BedrockModel(model_id=MODEL_ID)

model = BedrockModel(
                model_id=MODEL_ID,
                max_tokens=1024,
                temperature=0.8,
                top_k=50,
                top_p=0.95,
)

# Check model capabilities
supports_system_prompt = model_supports_system_prompt(MODEL_ID)
supports_tools = model_supports_tools(MODEL_ID)

# Build a static system prompt (rules only, no src embedded)
SYSTEM_PROMPT = get_analyzer_prompt("")

# Prepare agent configuration
agent_kwargs = {"model": model}

"""
if supports_tools:
    agent_kwargs["tools"] = [fetch_data]


if supports_system_prompt:
    agent_kwargs["system_prompt"] = SYSTEM_PROMPT

# Initialize agent once with the right capabilities
agent = Agent(**agent_kwargs)
"""

# Add tools if available
optional_tools = []
if fetch_data:
    optional_tools.append(fetch_data)
if build_eval_plan_v2:
    optional_tools.append(build_eval_plan_v2)

if supports_tools and optional_tools:
    agent_kwargs["tools"] = optional_tools

if supports_system_prompt:
    agent_kwargs["system_prompt"] = SYSTEM_PROMPT

agent = Agent(**agent_kwargs)

# Initialize the AgentCore Runtime App
app = BedrockAgentCoreApp()  #### AGENTCORE RUNTIME - LINE 2 ####


@app.entrypoint  #### AGENTCORE RUNTIME - LINE 3 ####
def invoke(payload):
    """AgentCore Runtime entrypoint function"""
    try:
        user_input = payload.get("prompt", "").strip()
        if not user_input:
            return {
                "statusCode": 400,
                "body": json.dumps({"error": "No prompt provided."})
            }

        # Step 1: Run the agent
        if supports_tools and supports_system_prompt:
            response = agent(f"DATA_SOURCE: {user_input}")
        elif supports_tools:
            response = agent(f"{SYSTEM_PROMPT}\n\nDATA_SOURCE: {user_input}")
        elif supports_system_prompt:
            response = agent(f"DATA_SOURCE: {user_input}")
        else:
            response = agent(f"{SYSTEM_PROMPT}\n\nDATA_SOURCE: {user_input}")
            fetch_data(user_input)

        # Step 2: Parse agent output
        parsed = _coerce_json(response)

        # Step 3: Normalize smart goals
        smart_goals = []
        goals_data = parsed.get("smart_goals") or parsed.get("goals") or []

        for idx, goal in enumerate(goals_data, start=1):
            if isinstance(goal, dict):
                desc = goal.get("description") or goal.get("goal") or str(goal)
            else:
                desc = str(goal)
            smart_goals.append({
                "goal_number": idx,
                "description": desc.strip()
            })

        # Step 4: Final structured output
        output_obj = {
            "model_id": MODEL_ID,
            "data_source": user_input,
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
            "smart_goals": smart_goals,
        }

        # Step 5: Save outputs
        os.makedirs(OUTPUT_DIR_INDIVIDUAL, exist_ok=True)
        base = _basename_no_ext(user_input)
        safe_model = _safe_fragment(MODEL_ID)
        out_path = os.path.join(
            OUTPUT_DIR_INDIVIDUAL, f"{base}_{safe_model}_output.json"
        )

        with open(out_path, "w", encoding="utf-8") as f:
            json.dump(output_obj, f, ensure_ascii=False, indent=2)

        _append_jsonl(output_jsonl, output_obj)

        # Step 6: Call evaluator runtime (optional)
        evaluator_result = None
        if build_eval_plan_v2:
            try:
                output_obj1 = output_obj
                payload={'analyzer_payload':output_obj1}
                raw_output = call_evaluator_runtime(payload) 
                eval_dict = json.loads(raw_output['body'])['evaluator_output']
                evaluator_result = json.dumps(eval_dict, indent=2)
                
            except Exception as ex:
                print(f"Evaluator runtime failed: {ex}")
                evaluator_result = {"error": str(ex)}
       
        # Step 7: Return HTTP-style response
        combined = {"model_output": output_obj}
        if evaluator_result:
            combined["evaluator_result"] = evaluator_result

        return {
            "statusCode": 200,
            "headers": {"Content-Type": "application/json"},
            "body": json.dumps(combined, ensure_ascii=False),
        }
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return {
            "statusCode": 500,
            "body": json.dumps({"error": str(e)})
        }


    #user_input = payload.get("prompt", "")      #user_input is the file path
    # Invoke the agent
    #response = agent(user_input)
    #return response.message["content"][0]["text"]


if __name__ == "__main__":
    app.run()  #### AGENTCORE RUNTIME - LINE 4 ####



Overwriting ./lab_helpers/smartgoalgenerator_runtime.py


## Phase 3 - Preparing Cognito OAuth2 for Authorization to Lamdba from Gateway Request

In [16]:
from lab_helpers.utils import setup_cognito_user_pool, reauthenticate_user

print("Setting up Amazon Cognito user pool...")
cognito_config = (
    setup_cognito_user_pool()
)  # You'll get your bearer token from this output cell.
print("Cognito setup completed ✓")

Setting up Amazon Cognito user pool...
{'UserPoolId': 'us-east-1_HYWGixp4p', 'ClientName': 'MCPServerPoolClient', 'ClientId': '4id0g63er01tmgkohp7ql63g0q', 'ClientSecret': '1jq3lcfqa5d6e7rnn0dkpk6o2fkojuutbsngv55ek0k8bebo6mh2', 'LastModifiedDate': datetime.datetime(2025, 10, 16, 6, 12, 6, 253000, tzinfo=tzlocal()), 'CreationDate': datetime.datetime(2025, 10, 16, 6, 12, 6, 253000, tzinfo=tzlocal()), 'RefreshTokenValidity': 30, 'TokenValidityUnits': {}, 'ExplicitAuthFlows': ['ALLOW_USER_PASSWORD_AUTH', 'ALLOW_USER_SRP_AUTH', 'ALLOW_REFRESH_TOKEN_AUTH'], 'AllowedOAuthFlowsUserPoolClient': False, 'EnableTokenRevocation': True, 'EnablePropagateAdditionalUserContextData': False, 'AuthSessionValidity': 3}
Pool id: us-east-1_HYWGixp4p
Discovery URL: https://cognito-idp.us-east-1.amazonaws.com/us-east-1_HYWGixp4p/.well-known/openid-configuration
Client ID: 4id0g63er01tmgkohp7ql63g0q
Bearer Token: eyJraWQiOiJLcFpEbSs5bkhJamJZclZpK1ZkXC9IZlVMZDdQbllYU3F1cXVlamdXUkRvcz0iLCJhbGciOiJSUzI1NiJ9.eyJzdW

## Phase 3 - Install Agentcore toolkit and Create Execution Role

In [18]:
pip install bedrock-agentcore-starter-toolkit

Note: you may need to restart the kernel to use updated packages.


## Phase 3 - Create/Configure Agentcore Runtime as Docker and Deploy to ECR

In [19]:
from bedrock_agentcore_starter_toolkit import Runtime
from lab_helpers.utils import create_agentcore_runtime_execution_role

# Initialize the runtime toolkit
boto_session = boto3.session.Session()
region = boto_session.region_name

execution_role_arn = create_agentcore_runtime_execution_role()

agentcore_runtime = Runtime()

# Configure the deployment
response = agentcore_runtime.configure(
    entrypoint="lab_helpers/smartgoalgenerator_runtime.py",
    execution_role=execution_role_arn,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="smart_goal_generator_agent",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [cognito_config.get("client_id")],
            "discoveryUrl": cognito_config.get("discovery_url"),
        }
    },
)

print("Configuration completed:", response)

Entrypoint parsed: file=/mnt/custom-file-systems/efs/fs-09f36259b5e98907e_fsap-09cbf9f8e29ef1a0c/SIPPA-smart-goal-generator/lab_helpers/smartgoalgenerator_runtime.py, bedrock_agentcore_name=smartgoalgenerator_runtime
Memory configured with STM only
Configuring BedrockAgentCore agent: smart_goal_generator_agent


ℹ️ Role SmartGoalGeneratorBedrockAgentCoreRole-us-east-1 already exists
Role ARN: arn:aws:iam::711246752798:role/SmartGoalGeneratorBedrockAgentCoreRole-us-east-1


💡 No container engine found (Docker/Finch/Podman not installed)

✓ Default deployment uses CodeBuild (no container engine needed), For local builds, install Docker, Finch, or 
Podman

Will create new memory with mode: STM_ONLY
Memory configuration: Short-term memory only
Found existing memory ID from previous launch: smart_goal_generator_agent_mem-7XGgEH7GBM


⚠️ Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64', so local builds
won't work.
Please use default launch command which will do a remote cross-platform build using code build.For deployment other
options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: Dockerfile
Generated .dockerignore: /mnt/custom-file-systems/efs/fs-09f36259b5e98907e_fsap-09cbf9f8e29ef1a0c/SIPPA-smart-goal-generator/.dockerignore
Keeping 'smart_goal_generator_agent' as default agent
Bedrock AgentCore configured: /mnt/custom-file-systems/efs/fs-09f36259b5e98907e_fsap-09cbf9f8e29ef1a0c/SIPPA-smart-goal-generator/.bedrock_agentcore.yaml


Configuration completed: config_path=PosixPath('/mnt/custom-file-systems/efs/fs-09f36259b5e98907e_fsap-09cbf9f8e29ef1a0c/SIPPA-smart-goal-generator/.bedrock_agentcore.yaml') dockerfile_path=PosixPath('/mnt/custom-file-systems/efs/fs-09f36259b5e98907e_fsap-09cbf9f8e29ef1a0c/SIPPA-smart-goal-generator/Dockerfile') dockerignore_path=PosixPath('/mnt/custom-file-systems/efs/fs-09f36259b5e98907e_fsap-09cbf9f8e29ef1a0c/SIPPA-smart-goal-generator/.dockerignore') runtime='None' region='us-east-1' account_id='711246752798' execution_role='arn:aws:iam::711246752798:role/SmartGoalGeneratorBedrockAgentCoreRole-us-east-1' ecr_repository=None auto_create_ecr=True memory_id=None


In [20]:
# Launch the agent (this will build and deploy the container)
from lab_helpers.utils import put_ssm_parameter

launch_result = agentcore_runtime.launch()
print("Launch completed:", launch_result.agent_arn)

agent_arn = put_ssm_parameter(
    "/app/smartgoalgenerator/agentcore/runtime_arn", launch_result.agent_arn
)

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Creating memory resource for agent: smart_goal_generator_agent
✅ MemoryManager initialized for region: us-east-1
🔎 Retrieving memory resource with ID: smart_goal_generator_agent_mem-7XGgEH7GBM...
  Found memory: smart_goal_generator_agent_mem-7XGgEH7GBM
Found existing memory in cloud: smart_goal_generator_agent_mem-7XGgEH7GBM
Existing memory has 0 strategies
✅ Using existing STM-only memory
Starting CodeBuild ARM64 deployment for agent 'smart_goal_generator_agent' to account 711246752798 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository f

✅ Reusing existing ECR repository: 711246752798.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-smart_goal_generator_agent


Reusing existing CodeBuild execution role: arn:aws:iam::711246752798:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-982ee6f783
Using dockerignore.template with 45 patterns for zip filtering
Uploaded source to S3: smart_goal_generator_agent/source.zip
Updated CodeBuild project: bedrock-agentcore-smart_goal_generator_agent-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.0s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 9.3s
🔄 DOWNLOAD_SOURCE started (total: 10s)
✅ DOWNLOAD_SOURCE completed in 2.1s
🔄 BUILD started (total: 12s)
✅ BUILD completed in 16.5s
🔄 POST_BUILD started (total: 29s)
✅ POST_BUILD completed in 12.4s
🔄 FINALIZING started (total: 41s)
✅ FINALIZING completed in 1.0s
🔄 COMPLETED started (total: 42s)
✅ COMPLETED completed in 1.0s
🎉 CodeBuild completed successfully in 0m 43s
CodeBuild completed successfully
✅ CodeBuild project configuration saved
Deploy

Launch completed: arn:aws:bedrock-agentcore:us-east-1:711246752798:runtime/smart_goal_generator_agent-YAsRgN3gAq


## Phase 3 Milestone 1 Completed: Agentcore Agent Runtime Deployment to ECR Integrated with MCP tools Ready. Time to Check Provisioning Status 

In [23]:
import time

# Wait for the agent to be ready
status_response = agentcore_runtime.status()
status = status_response.endpoint["status"]

end_status = ["READY", "CREATE_FAILED", "DELETE_FAILED", "UPDATE_FAILED"]
while status not in end_status:
    print(f"Waiting for deployment... Current status: {status}")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint["status"]

print(f"Final status: {status}")

✅ MemoryManager initialized for region: us-east-1
🔎 Retrieving memory resource with ID: smart_goal_generator_agent_mem-7XGgEH7GBM...
  Found memory: smart_goal_generator_agent_mem-7XGgEH7GBM
Retrieved Bedrock AgentCore status for: smart_goal_generator_agent


Final status: READY


## Phase 3 Milestone 2 - Invoke Agentcore Agents in Two Different Kernel Sessions. 

### Agent 1 (in this session): Invoke MCP Tool via API Gateway to Retrieve a Medical Record and Generate S.M.A.R.T. goals According to the Medical Record 

### Agent 2 (in a separated session): Act as LLM-as-Judge to Evaluate the S.M.A.R.T. Goals According to Defined Metrics 

In [38]:
import uuid

# Create a session ID for demonstrating session continuity
session_id = uuid.uuid4()

# Test different customer support scenarios
#user_query = "s3://patient-summary-bucket/patient1_summary.docx"
user_query = "s3://sippa/app_data_repo/SIPPA_AI-Extraction-Treatment-Plan/clinician_summary_data_source/AM-09121152.docx"


bearer_token = reauthenticate_user(
    cognito_config.get("client_id"), 
    cognito_config.get("client_secret")
)

response = agentcore_runtime.invoke(
    {"prompt": user_query}, 
    bearer_token=bearer_token,
    session_id=str(session_id)
)
response

{'response': '{"statusCode": 200, "headers": {"Content-Type": "application/json"}, "body": "{\\"model_output\\": {\\"model_id\\": \\"mistral.mistral-7b-instruct-v0:2\\", \\"data_source\\": \\"s3://sippa/app_data_repo/SIPPA_AI-Extraction-Treatment-Plan/clinician_summary_data_source/AM-09121152.docx\\", \\"timestamp\\": \\"2025-10-16 06:38:53\\", \\"smart_goals\\": [{\\"goal_number\\": 1, \\"description\\": \\"Reduce daily carbohydrate intake to 45g per meal, as per the meal plan provided by the healthcare team. Monitor progress by recording and tracking carbohydrate intake daily using a food diary app, aiming for an average of 135g carbohydrates per day. Review progress with healthcare team every 2 weeks.\\"}, {\\"goal_number\\": 2, \\"description\\": \\"Engage in at least 150 minutes of moderate-intensity aerobic activity or 75 minutes of vigorous-intensity aerobic activity per week, as recommended by the American Diabetes Association. Track progress using a fitness tracker and record 

### Phase 3 Wrapped Up. Log the Session Activity

In [31]:
import boto3
print(boto3.client("sts").get_caller_identity())



{'UserId': 'AROA2LGM4FQPKRBVPP2VG:SageMaker', 'Account': '711246752798', 'Arn': 'arn:aws:sts::711246752798:assumed-role/AmazonSageMaker-ExecutionRole-20250818T210256/SageMaker', 'ResponseMetadata': {'RequestId': '13e4dcd0-db5a-4361-9ff6-df7ad371f414', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '13e4dcd0-db5a-4361-9ff6-df7ad371f414', 'x-amz-sts-extended-request-id': 'MTp1cy1lYXN0LTE6MTc2MDU5NjYwMjYzNjpSOlFHN2RISmJ5', 'content-type': 'text/xml', 'content-length': '470', 'date': 'Thu, 16 Oct 2025 06:36:42 GMT'}, 'RetryAttempts': 0}}


## Unit Testing on Accessing Agentcore Agent Residing in a Different Session

In [32]:
import boto3
import json

EVALUATOR_RUNTIME_ARN = "arn:aws:bedrock-agentcore:us-east-1:711246752798:runtime/llm_evaluator_agent-noHKUN9oQS"

def call_evaluator_runtime(payload: dict) -> dict:
    # Initialize the Bedrock AgentCore client
    agent_core_client = boto3.client('bedrock-agentcore')
  
    # Prepare the payload prompt
    #payload={'analyzer_payload':output_obj}
    prompt = json.dumps(payload).encode()
  
    # Invoke the agent
    response = agent_core_client.invoke_agent_runtime(
                    agentRuntimeArn=EVALUATOR_RUNTIME_ARN,
                    #agentRuntimeArn="arn:aws:bedrock-agentcore:us-east-1:711246752798:runtime/llm_evaluator_agent-jf0YsKAH8C", 
                    #runtimeSessionId=session_id,
                    payload=prompt
                    )

    # Process and print the response
    if "text/event-stream" in response.get("contentType", ""):
        # Handle streaming response
        content = []
        for line in response["response"].iter_lines(chunk_size=10):
            if line:
                line = line.decode("utf-8")
                if line.startswith("data: "):
                    line = line[6:]
                    print(line)
                    content.append(line)
        #print("\nComplete response:", "\n".join(content))
        return "\n".join(content)
        
    elif response.get("contentType") == "application/json":
        # Handle standard JSON response
        content = []
        for chunk in response.get("response", []):
            content.append(chunk.decode('utf-8'))
        #print(json.loads(''.join(content)))
        return json.loads(''.join(content))
  
    #else:
        # Print raw response for other content types
        #print(response)
    return response


In [33]:
output_obj=[{'case_id': 1,
  'timestamp': '2025-10-13 00:55:59',
  'goal_number': 1,
  'goal_text': 'Reduce daily carbohydrate intake to 45g per meal and aim for a total of 135g per day. Monitor progress weekly by recording carbohydrate counts in food diary.'},
 {'case_id': 2,
  'timestamp': '2025-10-13 00:55:59',
  'goal_number': 2,
  'goal_text': 'Engage in at least 30 minutes of moderate-intensity aerobic activity 5 days per week. Track activity using a fitness tracker or smartphone app.'},
 {'case_id': 3,
  'timestamp': '2025-10-13 00:55:59',
  'goal_number': 3,
  'goal_text': 'Take metformin 500mg twice daily with meals. Refill prescription every 90 days and report any side effects to healthcare provider.'},
 {'case_id': 4,
  'timestamp': '2025-10-13 00:55:59',
  'goal_number': 4,
  'goal_text': 'Check blood glucose levels before meals and at bedtime. Target pre-meal blood glucose levels below 130mg/dL and bedtime levels below 110mg/dL. Record results in a log.'},
 {'case_id': 5,
  'timestamp': '2025-10-13 00:55:59',
  'goal_number': 5,
  'goal_text': 'Maintain adequate water intake by drinking at least 8 glasses of water per day. Monitor urine color and report any unusual changes to healthcare provider.'},
 {'case_id': 6,
  'timestamp': '2025-10-13 00:55:59',
  'goal_number': 6,
  'goal_text': 'Attend all scheduled appointments with healthcare provider and bring a copy of the most recent blood glucose log. Update medication list and share any medication changes.'}]

In [34]:
payload={'analyzer_payload':output_obj}

raw_output = call_evaluator_runtime(payload)

raw_output

{'statusCode': 200,
 'headers': {'Content-Type': 'application/json'},
 'body': '{"run_id": "e2f70b50-9d5f-4be9-a904-a104bec3b733", "timestamp": "2025-10-16 06:37:47", "evaluator_output": {"evaluation_type": "smart_goals_rubric", "cases_scored": 6, "scores": [{"case_id": 1, "metric_scores": {"specific": 1.0, "measurable": 1.0, "achievable": 0.9, "relevant": 1.0, "time_bound": 0.8, "clarity": 1.0}, "agreement": "n/a", "notes": "Clear carb targets (45g/meal, 135g/day) with weekly monitoring. Time-bound for monitoring but not end goal achievement."}, {"case_id": 2, "metric_scores": {"specific": 1.0, "measurable": 1.0, "achievable": 1.0, "relevant": 1.0, "time_bound": 1.0, "clarity": 1.0}, "agreement": "n/a", "notes": "Fully SMART with precise activity duration (30 min), frequency (5 days/week), intensity level, and tracking method specified."}, {"case_id": 3, "metric_scores": {"specific": 1.0, "measurable": 0.9, "achievable": 1.0, "relevant": 1.0, "time_bound": 0.9, "clarity": 1.0}, "agree

In [35]:
eval_output=json.loads(raw_output['body'])['evaluator_output']
eval_output

{'evaluation_type': 'smart_goals_rubric',
 'cases_scored': 6,
 'scores': [{'case_id': 1,
   'metric_scores': {'specific': 1.0,
    'measurable': 1.0,
    'achievable': 0.9,
    'relevant': 1.0,
    'time_bound': 0.8,
    'clarity': 1.0},
   'agreement': 'n/a',
   'notes': 'Clear carb targets (45g/meal, 135g/day) with weekly monitoring. Time-bound for monitoring but not end goal achievement.'},
  {'case_id': 2,
   'metric_scores': {'specific': 1.0,
    'measurable': 1.0,
    'achievable': 1.0,
    'relevant': 1.0,
    'time_bound': 1.0,
    'clarity': 1.0},
   'agreement': 'n/a',
   'notes': 'Fully SMART with precise activity duration (30 min), frequency (5 days/week), intensity level, and tracking method specified.'},
  {'case_id': 3,
   'metric_scores': {'specific': 1.0,
    'measurable': 0.9,
    'achievable': 1.0,
    'relevant': 1.0,
    'time_bound': 0.9,
    'clarity': 1.0},
   'agreement': 'n/a',
   'notes': 'Clear medication regimen with specific dosage, timing, and follow-up a

In [36]:
evaluator_result = json.dumps(eval_output, indent=2)
evaluator_result

'{\n  "evaluation_type": "smart_goals_rubric",\n  "cases_scored": 6,\n  "scores": [\n    {\n      "case_id": 1,\n      "metric_scores": {\n        "specific": 1.0,\n        "measurable": 1.0,\n        "achievable": 0.9,\n        "relevant": 1.0,\n        "time_bound": 0.8,\n        "clarity": 1.0\n      },\n      "agreement": "n/a",\n      "notes": "Clear carb targets (45g/meal, 135g/day) with weekly monitoring. Time-bound for monitoring but not end goal achievement."\n    },\n    {\n      "case_id": 2,\n      "metric_scores": {\n        "specific": 1.0,\n        "measurable": 1.0,\n        "achievable": 1.0,\n        "relevant": 1.0,\n        "time_bound": 1.0,\n        "clarity": 1.0\n      },\n      "agreement": "n/a",\n      "notes": "Fully SMART with precise activity duration (30 min), frequency (5 days/week), intensity level, and tracking method specified."\n    },\n    {\n      "case_id": 3,\n      "metric_scores": {\n        "specific": 1.0,\n        "measurable": 0.9,\n       

In [37]:
# Putting it together
output_obj1 = output_obj
payload={'analyzer_payload':output_obj1}
raw_output = call_evaluator_runtime(payload) 
eval_dict = json.loads(raw_output['body'])['evaluator_output']
evaluator_result = json.dumps(eval_dict, indent=2)
evaluator_result

'{\n  "evaluation_type": "smart_goals_rubric",\n  "cases_scored": 6,\n  "scores": [\n    {\n      "case_id": 1,\n      "metric_scores": {\n        "specific": 1.0,\n        "measurable": 1.0,\n        "achievable": 0.9,\n        "relevant": 1.0,\n        "time_bound": 0.8,\n        "clarity": 1.0\n      },\n      "agreement": "n/a",\n      "notes": "Goal specifies exact carb amounts (45g/meal, 135g/day) with weekly monitoring via food diary. Time frame for achievement could be more explicit."\n    },\n    {\n      "case_id": 2,\n      "metric_scores": {\n        "specific": 1.0,\n        "measurable": 1.0,\n        "achievable": 1.0,\n        "relevant": 1.0,\n        "time_bound": 0.9,\n        "clarity": 1.0\n      },\n      "agreement": "n/a",\n      "notes": "Clearly defines activity type, duration (30 min), frequency (5 days/week), and tracking method. Only missing overall timeframe for goal achievement."\n    },\n    {\n      "case_id": 3,\n      "metric_scores": {\n        "spec